In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor, XGBClassifier
import matplotlib.pyplot as plt

# Đọc dữ liệu
data = pd.read_csv('./clean_big_data.csv')
data['DT'] = pd.to_datetime(data['DT'])
data = data.sort_values('DT')

# Tạo các đặc trưng từ dữ liệu quá khứ
def create_features(df, label, hours):
    df_new = df.copy()
    for hour in range(1, hours + 1):
        df_new[f'Temp_-{hour}h'] = df_new['Temp'].shift(hour)
        df_new[f'Humidity_-{hour}h'] = df_new['Humidity'].shift(hour)

    df_new['Hour'] = df_new['DT'].dt.hour
    df_new['DayOfWeek'] = df_new['DT'].dt.dayofweek

    df_new = df_new.dropna()
    X = df_new[[f'Temp_-{h}h' for h in range(1, hours + 1)] +
                [f'Humidity_-{h}h' for h in range(1, hours + 1)] +
                ['Hour', 'DayOfWeek']]
    y = df_new[label]
    return X, y

# Tạo đặc trưng cho dự đoán xác suất mưa, nhiệt độ và độ ẩm
hours_past = 4
X_rain, y_rain = create_features(data, 'Is Rain', hours_past)
X_temp, y_temp = create_features(data, 'Temp', hours_past)
X_humidity, y_humidity = create_features(data, 'Humidity', hours_past)

# Chia dữ liệu
X_train_rain, X_test_rain, y_train_rain, y_test_rain = train_test_split(X_rain, y_rain, test_size=0.2, random_state=42)
X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)
X_train_humidity, X_test_humidity, y_train_humidity, y_test_humidity = train_test_split(X_humidity, y_humidity, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler_rain = StandardScaler()
scaler_temp = StandardScaler()
scaler_humidity = StandardScaler()
X_train_rain_scaled = scaler_rain.fit_transform(X_train_rain)
X_test_rain_scaled = scaler_rain.transform(X_test_rain)
X_train_temp_scaled = scaler_temp.fit_transform(X_train_temp)
X_test_temp_scaled = scaler_temp.transform(X_test_temp)
X_train_humidity_scaled = scaler_humidity.fit_transform(X_train_humidity)
X_test_humidity_scaled = scaler_humidity.transform(X_test_humidity)

# Xây dựng và huấn luyện mô hình dự đoán xác suất mưa
model_rain = XGBClassifier(n_estimators=100, random_state=42)
model_rain.fit(X_train_rain_scaled, y_train_rain)

# Xây dựng và huấn luyện mô hình dự đoán nhiệt độ
model_temp = XGBRegressor(n_estimators=100, random_state=42)
model_temp.fit(X_train_temp_scaled, y_train_temp)

# Xây dựng và huấn luyện mô hình dự đoán độ ẩm
model_humidity = XGBRegressor(n_estimators=100, random_state=42)
model_humidity.fit(X_train_humidity_scaled, y_train_humidity)

# Hàm dự đoán cho nhiều giờ tiếp theo
def predict_weather_multiple(temp_history, humidity_history, hours_ahead):
    if len(temp_history) != hours_past or len(humidity_history) != hours_past:
        raise ValueError(f"Cần cung cấp {hours_past} giá trị nhiệt độ và độ ẩm gần nhất")

    predictions = []

    for _ in range(hours_ahead):
        input_data = np.array(temp_history + humidity_history + [0, 0]).reshape(1, -1)
        input_data_scaled_rain = scaler_rain.transform(input_data)
        input_data_scaled_temp = scaler_temp.transform(input_data)
        input_data_scaled_humidity = scaler_humidity.transform(input_data)

        rain_prob = model_rain.predict_proba(input_data_scaled_rain)[0][1]
        temp_forecast = model_temp.predict(input_data_scaled_temp)[0]
        humidity_forecast = model_humidity.predict(input_data_scaled_humidity)[0]

        predictions.append((temp_forecast, humidity_forecast, rain_prob))

        temp_history.append(temp_forecast)
        humidity_history.append(humidity_forecast)
        temp_history.pop(0)
        humidity_history.pop(0)

    return predictions

# Ví dụ sử dụng
temp_history = [28.62, 30.98, 28.56, 27]
humidity_history = [76, 79, 80, 78]
hours_ahead = 8

predictions = predict_weather_multiple(temp_history, humidity_history, hours_ahead)

print(f"\nDự đoán thời tiết cho {hours_ahead} giờ tiếp theo:")
for i, (temp, humidity, rain_prob) in enumerate(predictions):
    print(f"Giờ {i + 1}: Nhiệt độ dự đoán: {temp:.1f}°C, Độ ẩm dự đoán: {humidity:.1f}%, Xác suất mưa: {rain_prob:.2%}")



Dự đoán thời tiết cho 8 giờ tiếp theo:
Giờ 1: Nhiệt độ dự đoán: 28.0°C, Độ ẩm dự đoán: 79.8%, Xác suất mưa: 63.13%
Giờ 2: Nhiệt độ dự đoán: 30.8°C, Độ ẩm dự đoán: 73.4%, Xác suất mưa: 69.79%
Giờ 3: Nhiệt độ dự đoán: 28.1°C, Độ ẩm dự đoán: 79.8%, Xác suất mưa: 9.85%
Giờ 4: Nhiệt độ dự đoán: 27.6°C, Độ ẩm dự đoán: 76.9%, Xác suất mưa: 25.44%
Giờ 5: Nhiệt độ dự đoán: 28.6°C, Độ ẩm dự đoán: 80.1%, Xác suất mưa: 33.86%
Giờ 6: Nhiệt độ dự đoán: 31.2°C, Độ ẩm dự đoán: 68.5%, Xác suất mưa: 42.66%
Giờ 7: Nhiệt độ dự đoán: 28.0°C, Độ ẩm dự đoán: 80.5%, Xác suất mưa: 61.01%
Giờ 8: Nhiệt độ dự đoán: 28.4°C, Độ ẩm dự đoán: 82.3%, Xác suất mưa: 58.69%


c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScal

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Đọc dữ liệu
data = pd.read_csv('./clean_big_data.csv')
data['DT'] = pd.to_datetime(data['DT'])
data = data.sort_values('DT')

# Tạo các đặc trưng từ dữ liệu quá khứ
def create_features(df, label, hours):
    df_new = df.copy()
    for hour in range(1, hours + 1):
        df_new[f'Temp_-{hour}h'] = df_new['Temp'].shift(hour)
        df_new[f'Humidity_-{hour}h'] = df_new['Humidity'].shift(hour)

    df_new['Hour'] = df_new['DT'].dt.hour
    df_new['DayOfWeek'] = df_new['DT'].dt.dayofweek

    df_new = df_new.dropna()
    X = df_new[[f'Temp_-{h}h' for h in range(1, hours + 1)] +
                [f'Humidity_-{h}h' for h in range(1, hours + 1)] +
                ['Hour', 'DayOfWeek']]
    y = df_new[label]
    return X, y

# Tạo đặc trưng cho dự đoán xác suất mưa, nhiệt độ và độ ẩm
hours_past = 4
X_rain, y_rain = create_features(data, 'Is Rain', hours_past)
X_temp, y_temp = create_features(data, 'Temp', hours_past)
X_humidity, y_humidity = create_features(data, 'Humidity', hours_past)

# Chia dữ liệu
X_train_rain, X_test_rain, y_train_rain, y_test_rain = train_test_split(X_rain, y_rain, test_size=0.2, random_state=42)
X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)
X_train_humidity, X_test_humidity, y_train_humidity, y_test_humidity = train_test_split(X_humidity, y_humidity, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler_rain = StandardScaler()
scaler_temp = StandardScaler()
scaler_humidity = StandardScaler()
X_train_rain_scaled = scaler_rain.fit_transform(X_train_rain)
X_test_rain_scaled = scaler_rain.transform(X_test_rain)
X_train_temp_scaled = scaler_temp.fit_transform(X_train_temp)
X_test_temp_scaled = scaler_temp.transform(X_test_temp)
X_train_humidity_scaled = scaler_humidity.fit_transform(X_train_humidity)
X_test_humidity_scaled = scaler_humidity.transform(X_test_humidity)

# Xây dựng và huấn luyện mô hình dự đoán xác suất mưa
model_rain = XGBClassifier(n_estimators=100, random_state=42)
model_rain.fit(X_train_rain_scaled, y_train_rain)

# Xây dựng và huấn luyện mô hình dự đoán nhiệt độ
model_temp = XGBRegressor(n_estimators=100, random_state=42)
model_temp.fit(X_train_temp_scaled, y_train_temp)

# Xây dựng và huấn luyện mô hình dự đoán độ ẩm
model_humidity = XGBRegressor(n_estimators=100, random_state=42)
model_humidity.fit(X_train_humidity_scaled, y_train_humidity)

# Hàm dự đoán cho nhiều giờ tiếp theo
def predict_weather_multiple(temp_history, humidity_history, hours_ahead):
    if len(temp_history) != hours_past or len(humidity_history) != hours_past:
        raise ValueError(f"Cần cung cấp {hours_past} giá trị nhiệt độ và độ ẩm gần nhất")

    predictions = []

    for _ in range(hours_ahead):
        input_data = np.array(temp_history + humidity_history + [0, 0]).reshape(1, -1)
        input_data_scaled_rain = scaler_rain.transform(input_data)
        input_data_scaled_temp = scaler_temp.transform(input_data)
        input_data_scaled_humidity = scaler_humidity.transform(input_data)

        rain_prob = model_rain.predict_proba(input_data_scaled_rain)[0][1]
        temp_forecast = model_temp.predict(input_data_scaled_temp)[0]
        humidity_forecast = model_humidity.predict(input_data_scaled_humidity)[0]

        predictions.append((temp_forecast, humidity_forecast, rain_prob))

        temp_history.append(temp_forecast)
        humidity_history.append(humidity_forecast)
        temp_history.pop(0)
        humidity_history.pop(0)

    return predictions

# Hàm đánh giá mô hình xác suất mưa
def evaluate_rain_model(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print("Đánh giá mô hình xác suất mưa:")
    print(f"Độ chính xác: {accuracy:.2f}")
    print(f"Độ nhạy (Recall): {recall:.2f}")
    print(f"Độ chính xác (Precision): {precision:.2f}")
    print(f"F1 Score: {f1:.2f}")

# Hàm đánh giá mô hình nhiệt độ và độ ẩm
def evaluate_temp_humidity_model(y_true_temp, y_pred_temp, y_true_humidity, y_pred_humidity):
    # Đánh giá mô hình nhiệt độ
    mae_temp = mean_absolute_error(y_true_temp, y_pred_temp)
    mse_temp = mean_squared_error(y_true_temp, y_pred_temp)
    r2_temp = r2_score(y_true_temp, y_pred_temp)

    print("\nĐánh giá mô hình nhiệt độ:")
    print(f"Mean Absolute Error: {mae_temp:.2f}")
    print(f"Mean Squared Error: {mse_temp:.2f}")
    print(f"R-squared: {r2_temp:.2f}")

    # Đánh giá mô hình độ ẩm
    mae_humidity = mean_absolute_error(y_true_humidity, y_pred_humidity)
    mse_humidity = mean_squared_error(y_true_humidity, y_pred_humidity)
    r2_humidity = r2_score(y_true_humidity, y_pred_humidity)

    print("\nĐánh giá mô hình độ ẩm:")
    print(f"Mean Absolute Error: {mae_humidity:.2f}")
    print(f"Mean Squared Error: {mse_humidity:.2f}")
    print(f"R-squared: {r2_humidity:.2f}")


y_true_rain = y_test_rain
y_pred_rain = model_rain.predict(X_test_rain_scaled)

y_true_temp = y_test_temp
y_pred_temp = model_temp.predict(X_test_temp_scaled)

y_true_humidity = y_test_humidity
y_pred_humidity = model_humidity.predict(X_test_humidity_scaled)

evaluate_rain_model(y_true_rain, y_pred_rain)
evaluate_temp_humidity_model(y_true_temp, y_pred_temp, y_true_humidity, y_pred_humidity)

temp_history = [25, 24, 23, 22]
humidity_history = [70, 75, 80, 78]
hours_ahead = 8

predictions = predict_weather_multiple(temp_history, humidity_history, hours_ahead)

print(f"\nDự đoán thời tiết cho {hours_ahead} giờ tiếp theo:")
for i, (temp, humidity, rain_prob) in enumerate(predictions):
    print(f"Giờ {i + 1}: Nhiệt độ dự đoán: {temp:.1f}°C, Độ ẩm dự đoán: {humidity:.1f}%, Xác suất mưa: {rain_prob:.2%}")


Đánh giá mô hình xác suất mưa:
Độ chính xác: 0.77
Độ nhạy (Recall): 0.70
Độ chính xác (Precision): 0.74
F1 Score: 0.72

Đánh giá mô hình nhiệt độ:
Mean Absolute Error: 0.85
Mean Squared Error: 2.00
R-squared: 0.84

Đánh giá mô hình độ ẩm:
Mean Absolute Error: 4.50
Mean Squared Error: 44.11
R-squared: 0.69

Dự đoán thời tiết cho 8 giờ tiếp theo:
Giờ 1: Nhiệt độ dự đoán: 24.5°C, Độ ẩm dự đoán: 71.6%, Xác suất mưa: 0.62%
Giờ 2: Nhiệt độ dự đoán: 23.7°C, Độ ẩm dự đoán: 75.9%, Xác suất mưa: 2.94%
Giờ 3: Nhiệt độ dự đoán: 23.0°C, Độ ẩm dự đoán: 72.9%, Xác suất mưa: 2.90%
Giờ 4: Nhiệt độ dự đoán: 22.1°C, Độ ẩm dự đoán: 76.8%, Xác suất mưa: 0.22%
Giờ 5: Nhiệt độ dự đoán: 23.7°C, Độ ẩm dự đoán: 74.5%, Xác suất mưa: 0.98%
Giờ 6: Nhiệt độ dự đoán: 24.5°C, Độ ẩm dự đoán: 73.7%, Xác suất mưa: 0.93%
Giờ 7: Nhiệt độ dự đoán: 23.0°C, Độ ẩm dự đoán: 75.0%, Xác suất mưa: 1.90%
Giờ 8: Nhiệt độ dự đoán: 21.5°C, Độ ẩm dự đoán: 77.6%, Xác suất mưa: 1.24%


c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Tranv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScal

In [4]:
from joblib import dump, load

In [5]:
dump(model_rain, 'model_rain.joblib')
dump(model_temp, 'model_temp.joblib')
dump(model_humidity, 'model_humidity.joblib')

dump(scaler_rain, 'scaler_rain.joblib')
dump(scaler_temp, 'scaler_temp.joblib')
dump(scaler_humidity, 'scaler_humidity.joblib')


['scaler_humidity.joblib']

In [6]:

from joblib import load

# Tải mô hình
model_rain = load('model_rain.joblib')
model_temp = load('model_temp.joblib')
model_humidity = load('model_humidity.joblib')

# Tải scaler
scaler_rain = load('scaler_rain.joblib')
scaler_temp = load('scaler_temp.joblib')
scaler_humidity = load('scaler_humidity.joblib')



In [7]:
import numpy as np
import pandas as pd
temp_history = [25, 24, 23, 22]  # Nhiệt độ 4 giờ gần nhất
humidity_history = [70, 75, 80, 78]  # Độ ẩm 4 giờ gần nhất
hours_ahead = 8
hours_past = 4
# Hàm dự đoán cho nhiều giờ tiếp theo
def predict_weather_multiple(temp_history, humidity_history, hours_ahead):
    if len(temp_history) != hours_past or len(humidity_history) != hours_past:
        raise ValueError(f"Cần cung cấp {hours_past} giá trị nhiệt độ và độ ẩm gần nhất")

    predictions = []

    for _ in range(hours_ahead):
        input_data = np.array(temp_history + humidity_history + [0, 0]).reshape(1, -1)

        # Tạo DataFrame với tên cột
        feature_names = [f'Temp_-{h}h' for h in range(1, hours_past + 1)] + \
                        [f'Humidity_-{h}h' for h in range(1, hours_past + 1)] + \
                        ['Hour', 'DayOfWeek']

        input_data_df = pd.DataFrame(input_data, columns=feature_names)

        input_data_scaled_rain = scaler_rain.transform(input_data_df)
        input_data_scaled_temp = scaler_temp.transform(input_data_df)
        input_data_scaled_humidity = scaler_humidity.transform(input_data_df)

        rain_prob = model_rain.predict_proba(input_data_scaled_rain)[0][1]
        temp_forecast = model_temp.predict(input_data_scaled_temp)[0]
        humidity_forecast = model_humidity.predict(input_data_scaled_humidity)[0]

        predictions.append((temp_forecast, humidity_forecast, rain_prob))

        temp_history.append(temp_forecast)
        humidity_history.append(humidity_forecast)
        temp_history.pop(0)
        humidity_history.pop(0)

    return predictions

predictions = predict_weather_multiple(temp_history, humidity_history, hours_ahead)
print(f"\nDự đoán thời tiết cho {hours_ahead} giờ tiếp theo:")
for i, (temp, humidity, rain_prob) in enumerate(predictions):
    print(f"Giờ {i + 1}: Nhiệt độ dự đoán: {temp:.1f}°C, Độ ẩm dự đoán: {humidity:.1f}%, Xác suất mưa: {rain_prob:.2%}")



Dự đoán thời tiết cho 8 giờ tiếp theo:
Giờ 1: Nhiệt độ dự đoán: 24.5°C, Độ ẩm dự đoán: 71.6%, Xác suất mưa: 0.62%
Giờ 2: Nhiệt độ dự đoán: 23.7°C, Độ ẩm dự đoán: 75.9%, Xác suất mưa: 2.94%
Giờ 3: Nhiệt độ dự đoán: 23.0°C, Độ ẩm dự đoán: 72.9%, Xác suất mưa: 2.90%
Giờ 4: Nhiệt độ dự đoán: 22.1°C, Độ ẩm dự đoán: 76.8%, Xác suất mưa: 0.22%
Giờ 5: Nhiệt độ dự đoán: 23.7°C, Độ ẩm dự đoán: 74.5%, Xác suất mưa: 0.98%
Giờ 6: Nhiệt độ dự đoán: 24.5°C, Độ ẩm dự đoán: 73.7%, Xác suất mưa: 0.93%
Giờ 7: Nhiệt độ dự đoán: 23.0°C, Độ ẩm dự đoán: 75.0%, Xác suất mưa: 1.90%
Giờ 8: Nhiệt độ dự đoán: 21.5°C, Độ ẩm dự đoán: 77.6%, Xác suất mưa: 1.24%
